In [ ]:
######

### Variational Quantum Circuit

VQC is a parameterized model consisting of quantum gates and entanglement operations. These circuits are designed to encode data into quantum states and then perform operations that can reveal patterns useful for classification.

In [3]:
#Import necessary libraries
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import pennylane as qml
from pennylane import numpy as pnp
import pandas as pd
from pennylane.optimize import NesterovMomentumOptimizer
import math

'''Data preparation: This involves data loading, split, encoding 
the data with appropriate label and standardizing/normalizing the data'''
# Load the Breast Cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Encode the labels to {-1, 1}
le = LabelEncoder()
y = le.fit_transform(y) * 2 - 1

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

'''N = Number of qubits

 Here we have to make sure that the number of qubits are log2(n), where n = number of features.

 If we were using Angle embedding we would have used same number of qubits as number of features.'''
N=5 

if N**2 <= X_train.shape[1]:
    num_features = N **2
else:
    num_features = X_train.shape[1]

X_train=X_train[ : , :num_features]
X_test=X_test[ : , : num_features]

'''Creates quantum device that will run our circuits. Here we use 'default.qubit' Pennylane quantum circuit simulator'''
# Define a device
dev = qml.device("default.qubit", wires=N)


'''Define the quantum circuit.
Here we define a parameterized quantum circuit (Ansatz). We use a simple variational circuit:
'''
@qml.qnode(dev)
def variational_circuit(params, x):
    qml.templates.AmplitudeEmbedding(x, wires=range(N), normalize=True, pad_with=0.0)
    qml.templates.StronglyEntanglingLayers(params, wires=range(N))
    return qml.expval(qml.PauliZ(0))


def variational_classifier(params, bias, x):
    return variational_circuit(params, x) + bias

'''We need a cost function to optimize. This function calculates the loss based on 
the current circuit parameters and predictions. This create a loss function to be minimized'''
# Define the cost function
def cost(params, bias, X, y):
    predictions = pnp.array([variational_classifier(params, bias, x) for x in X])
    return pnp.mean((predictions - y) ** 2)

'''Defining accuracy to evaluate performance'''
#Define accuracy
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc
 
'''Optimization step involves setting up the initial parameters.
We then use classical optimizers to adjust the parameters of the quantum 
circuit to minimizet the cost function. 
We can use optimizers gradient descent optimizers or more advanced optimizers like 
Stochastic gradient descent (Adam) optimizer, NeterovMomentumOptimizer to update the hyperparameters
'''
# Initialize the parameters
num_layers = 5
params = pnp.random.uniform(low=0, high=2 * np.pi, size=(num_layers, N , 3), requires_grad=True)
bias = pnp.array(0.0, requires_grad=True)
# Set up the optimizer
opt = NesterovMomentumOptimizer(0.3)
epochs = 150

'''We run optimization loop to update the hyperparameters'''
# Training loop
for epoch in range(epochs):
    params, bias = opt.step(lambda p, b: cost(p, b, X_train, y_train), params, bias)
    # params, bias = opt.step(cost, params, bias, X_train, y_train)
    # print(params)
    if (epoch + 1) % 5 == 0:            
        current_cost = cost(params, bias, X_train, y_train)
        print(f"Epoch {epoch + 1}: Cost = {current_cost}")

'''Finally we use the predictions made using learned model to find the accuracy '''
# Test the trained model
predictions = pnp.array([np.sign(variational_classifier(params, bias, x)) for x in X_test])
accuracy = pnp.mean(predictions == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
# acc.append([layers, step, round(accuracy.item(), 4)])
# print(acc)

Epoch 5: Cost = 0.9003227437604365
Epoch 10: Cost = 0.8319896373828645
Epoch 15: Cost = 0.7978646128268697
Epoch 20: Cost = 0.7810886638506145
Epoch 25: Cost = 0.7688536308257489
Epoch 30: Cost = 0.7575372167375625
Epoch 35: Cost = 0.7446157923528166
Epoch 40: Cost = 0.735726071392138
Epoch 45: Cost = 0.7301319706020605
Epoch 50: Cost = 0.7255258808790681
Epoch 55: Cost = 0.721812099820872
Epoch 60: Cost = 0.7191745756759761
Epoch 65: Cost = 0.716785177703814
Epoch 70: Cost = 0.7142318615452196
Epoch 75: Cost = 0.711690429107643
Epoch 80: Cost = 0.709382512513614
Epoch 85: Cost = 0.707348403272641
Epoch 90: Cost = 0.7054600053397403
Epoch 95: Cost = 0.7036429688732763
Epoch 100: Cost = 0.7019948650964521
Epoch 105: Cost = 0.7006778881227044
Epoch 110: Cost = 0.6997518130000672
Epoch 115: Cost = 0.6991336877670099
Epoch 120: Cost = 0.6986992165324531
Epoch 125: Cost = 0.698367090278164
Epoch 130: Cost = 0.6980970032775823
Epoch 135: Cost = 0.6978667678746505
Epoch 140: Cost = 0.69766311